Описание таблицы orders:
* user_id (string) - уникальный идентификатор пользователя
* order_id (string) - уникальный идентификатор заказа
* order_time (Int64) - время сделанного заказа в unixtime (секундны, UTC)
* order_cost (float) - стоимость заказа
* success_order_flg (bool) - идентификатор, определяющий был ли заказ успешно доставлен. Выполненный заказ - это заказ у которого в данной таблице success_order_flg=True

Пользователь считается привлеченным в день date, если он совершает первый заказ в день date.

Пользователь считается реактивированным в день date, если он совершает заказ в день date, при этом не совершил ни одного заказа в течении предшествующих 90 дней, но совершал хотя бы 1 заказ за свою пользовательскую историю ранее.

На выходе требуется получить таблицу:

* date - календарная дата привлечения или реактивации
* gmv360d_new - общая стоимость заказов за 360 дней после дня date, совершенных пользователями, привлеченными в день date
* gmv360d_reactivated - общая стоимость заказов за 360 дней после дня date, совершенных пользователями, реактивированными в день date
* users_count_new - кол-во пользователей, привлеченных в день date
* users_count_reactivated- кол-во пользователей, реактивированных в день date

Заказы, совершенные пользователем после очередной реактивации, не должны учитываться в расчете GMV для предыдущих периодов активности пользователя. Т.е., например, если пользователь пришел в день 0, совершал заказы по день 30, не совершал заказов до дня 130 и совершил заказ в день 130, его заказы в дни 130-490 должны учитываться только в gmv360d_reactivated для дня 130, но не в gmv360d_new дня 0.

То же правило распространяется на повторные реактивации пользователя. То есть если у рассмотренного нами пользователя на 230-й день жизни снова произошла реактивация, то gmv360d_reactivated будет посчитано для предыдущего периода за дни 130-229, а для итогового, с реактивацией на 230-й день, это поле будет рассчитано за дни 230-590.

Сколько пользователей было реактивировано 20 марта 2020 года?



###### 1. получаем данные, библиотеки, создаём функции для получения суммы заказов за год, а также привлечённых и активированных пользователей в конретные даты

In [1]:
import datetime
import pandas as pd
data = pd.read_csv('E:/ittensive/Data analytics path/Data processing tasks/2022-04-22.dataset.csv',  delimiter=";")

In [2]:
#функция получает срез фрейма между датами x и y,
def number_of_users(df, x, y):
    z = df[(df['date_of_attraction'] >= x) & (df['date_of_attraction'] <= y)]
    #а затем возращет количество пользователей по user_id
    count = df['user_id'].count()
    return count

#функция получения суммы между пероидами
#получает на вход фрейм данных, начальную и конечную дату
def sum_cost(df,x,y):
    #из этого фрейма беруться все данные, между начальной и конечной датой
    z = df[(df['date_of_attraction'] >= x) & (df['date_of_attraction'] <= y)]
    #получаем из этого фрейма сумму зачений с ценнами заказов
    sum_ = z['order_cost'].sum()
    return sum_

#функция создания списка занчений общих стоимостей заказов за 360 дней
#на вход нужно передать Dataframe, а также номера столбцов с начальной и конечной датой
def iterrows(df, fisrt, second):
    #создаём список сумм стоимостей заказов за 360 дней
    list_oredrs = []
    #получаем все строки нашего фрейма
    for row in df.itertuples():
        #берём значения из нужных нам колонок
        x = row[fisrt]
        y = row[second]
        #получаем значения из функции sum_cost()
        sum_ = sum_cost(df, x, y)
        #возвращаем итоговый список
        list_oredrs.append(sum_)
    return list_oredrs

In [4]:
#функция возращает количетсво пользователей в конкретную дату
#(дата является индексом в полученом фрейме, сгруппированном по дате, и реализовавшим метод "count()")
def count_users(df, x):
    count = df.loc[x][0]
    return count

#функция подсчёта пользователей на вход получает фрейм в котором нужно подсчитать пользователей, 
#и номер столбца в котором храниться user_id
def iterrows_users(df, number):
    #создаём список для формирования серии данных users_count_new
    list_users = []
    #получаем все строки фрейма
    for row in df.itertuples():
        #берём значения строки нужного нам столбца
        x = row[number]
        #передаём в функцию count_users дату и сгруппированный по дате фрейм,
        #и получаем количество пользователей привлечённых в этот день
        count = count_users(data_calling_user, x)
        list_users.append(count)
    return list_users

#функция получает на вход user_id
def date_reactivation_user(user):
    #получает по нему все данные из нашего основого фрейма данных
    frame = data.loc[(data['user_id'] == user)]
    #отсортируем их ро дате
    frame.sort_values("order_time")
    #убираем первый заказ этого пользователя 
    frame = frame.iloc [1: , :]
    #остоавляем только те заказы разница между которыми больше 89 дней, 23 часа, 59 минут, 59 секунд (7775999 секунд)
    frame =  frame[frame['order_time_diff'] > 7775999]
    return frame

In [48]:
#эта функция предназначенна для того, чтобы после объединения фреймов, 
#заполнить данными из фрейма  о реактивации, фрейм о привлечении клиентов, и наоборот
#на вход получает дату, и DateFrame из которого брать данные для заполнения
def count_users_rows(date, dataframe):
    user_new = 0.0
    user_reactivated = 0.0
    coust_new = 0
    coust_reactvidet = 0
    #получаем данные по конкретной дате
    dataframe  = dataframe.loc[dataframe['date'] == date]
    #если количетство привлечённых или реактиврованных пользователей
    #в каком-то из заказов больше нуля, этим значениям функция зполняет все остальные ячейки
    #этой серии данных в эту дату
    for row in dataframe.itertuples():
        if row[2] > 0:
            coust_new = row[2]
        if row[2] > 0:
            user_new += 1 
        if row[4] > 0:
            coust_reactvidet = row[4]
        if row[5] > 0:
            user_reactivated  = row[5]
    #меняем значения в нашем конеченом фрейме данных из которого мы будем брать ответ
    df_concat.loc[(df_concat['date'] == date), 'users_count_new'] = user_new
    df_concat.loc[(df_concat['date'] == date), 'users_count_reactivated'] = user_reactivated
    df_concat.loc[(df_concat['date'] == date), 'gmv360d_new'] = coust_new
    df_concat.loc[(df_concat['date'] == date), 'gmv360d_reactivated'] = coust_reactvidet

###### 2. Оставляем только данные об успешных заказах, сортируем их по пользователям и по дате, создаём серию данных с разницей во времени между заказами

In [6]:
data = data[data['success_order_flg'] == 1]
data.sort_values(["user_id", "order_time"], inplace=True)
data["order_time_diff"] = data["order_time"].diff()

In [7]:
#создаём серию данных, которая равна date + один год
data['gmv360d'] = data["order_time"] + 31104000


In [8]:
#так же нам понадобяться результаты суммы при группировки фрейма "по пользователям"
data_calling_cost = data.groupby("user_id").sum()

### Дата привлечения

###### 3.  Создаём новый DataFrame выпонляя группировку предыдущего "по пользователю" , выбирая минималные значения, это позволит нам взять данные для "первых дат" в которые пользователи совершали заказ, то есть получить данные в даты привлечения

In [9]:
data_calling = data.groupby("user_id").min()
data_calling = pd.DataFrame({"user_id": data_calling.index, "date_of_attraction": data_calling["order_time"],
                            'order_cost': data_calling_cost['order_cost'], 'gmv360d': data_calling['gmv360d']})
data_calling.reset_index(drop= True , inplace= True )


In [10]:
#получаем общую стоимость заказов за 360 дней после дня date, совершенных пользователями, привлеченными в день date
ex = iterrows(data_calling, 2, 4)
data_calling['gmv360d_new'] = ex

In [11]:
#преобразуем время из секунд UTC, в дату
data_calling['date_of_attraction'] = data_calling['date_of_attraction'].apply(lambda n: pd.to_datetime(n, unit='s'))
data_calling['date_of_attraction'] = pd.to_datetime(data_calling['date_of_attraction']).dt.date

In [12]:
#для того, чтобы узнать количество привелчённых пользователей в конретные даты, 
#нам нужно сгрупировать Фрейм по датам привлечения
data_calling_user = data_calling.groupby("date_of_attraction").count()

In [13]:
#удаляем ненужную нам в дальнейшем серию данных 
data_calling.drop('gmv360d', axis = 1, inplace = True)

In [14]:
#создаём серрию данных -  "кол-во пользователей, привлеченных в день date"
count = iterrows_users(data_calling, 2)
data_calling['users_count_new'] = count

In [15]:
#удалякм вторую не нужную нам серию данных
data_calling.drop('order_cost', axis = 1, inplace = True)

### Дата реактивации

###### 4. Для получения даты реактивации отбираем только те значения у которых с даты предыдщуго заказа прошло больше 90 дней 

In [16]:
#берём первую строку фрейма нащих данных, чтобы получить структуру фрейма 

dataframe = data.head(1)
#проходим список всех привлечённых пользователей
for user in data_calling['user_id']:
    #получаем его данные о реактивированных заказах если они есть
    user = date_reactivation_user(user)
    #создаём DataFrame из таких срезов
    dataframe = dataframe.append(user)
#удаляем первую строку, так как там данные "нереактивированного" заказа
data_reactivation = dataframe.dropna()
#создаём нужную нам структуру без лишних серий данных
data_reactivation = pd.DataFrame({"user_id": data_reactivation['user_id'], "date_reactivation": data_reactivation["order_time"],
                                  'order_cost': data_reactivation['order_cost'], 'gmv360d':data_reactivation['gmv360d']})

###### 5. получаем общую стоимость заказов за 360 дней после дня date, совершенных пользователями, реактивированными в день date

In [ ]:
#адптируем функцию sum_cost() под даные о реактивации
def sum_cost(df,x,y):
    z = df[(df['date_reactivation'] >= x) & (df['date_reactivation'] <= y)]
    sum_ = z['order_cost'].sum()
    return sum_
#получаем суммы заказов за года
ex = iterrows(data_reactivation, 2, 4)
data_reactivation['gmv360d_reactivated'] = ex
del ex

In [19]:
#переводим время из секунд UTC в дату
data_reactivation['date_reactivation'] = data_reactivation['date_reactivation'].apply(lambda n: pd.to_datetime(n ,unit='s'))
data_reactivation['date_reactivation'] = pd.to_datetime(data_reactivation['date_reactivation']).dt.date

In [20]:
#группируем фрейм по дате, для получения количество реактиврованных пользователей в каждую дату
data_reactivation_user = data_reactivation.groupby("date_reactivation").count()

In [21]:
#удаляем больше ненужную среию данных
data_reactivation.drop('gmv360d', axis = 1, inplace = True)

In [22]:
#iterrows_users адаптируем функцию для данных о реактивации

def iterrows_users(df, number):
    list_users = []
    for row in df.itertuples():
        x = row[number]
        count = count_users(data_reactivation_user, x)
        list_users.append(count)
    return list_users

In [23]:
#получаем количество реактивированных пользователей для каждого дня
count = iterrows_users(data_reactivation, 2)
data_reactivation['users_count_reactivated'] = count

In [24]:
#удаляем ненужныую серию данных
data_reactivation.drop('order_cost', axis = 1, inplace = True)

## Объединение фреймов

##### 1. изменям структуры Фреймов Данных для того, чтобы их было удобно объеденить

In [25]:
data_reactivation = pd.DataFrame({"user_id": data_reactivation['user_id'], "date": data_reactivation["date_reactivation"],
                                  'gmv360d_reactivated': data_reactivation['gmv360d_reactivated'], 
                                  'users_count_reactivated': data_reactivation['users_count_reactivated']})
data_calling = pd.DataFrame({"user_id": data_calling['user_id'], "date": data_calling["date_of_attraction"],
                                  'gmv360d_new': data_calling['gmv360d_new'], 
                                  'users_count_new': data_calling['users_count_new']})

In [26]:
#Назанчаем индексы по котроым будем объденять
data_reactivation = data_reactivation.set_index(['user_id', 'date'])
data_calling = data_calling.set_index(['user_id', 'date'])

###### 2. Объединяем фреймы

In [27]:
df_concat = pd.concat([ data_calling, data_reactivation],axis=1)

In [28]:
#заполняем отсутствующие значения нулями
df_concat.fillna(0, inplace = True)

In [29]:
#Сбрасываем идексы, чтобы можно было отсортировать данные по дате
df_concat = df_concat.reset_index()
df_concat.sort_values([ "date", 'user_id'], inplace=True)
df_concat.set_index('user_id', inplace = True)

In [30]:
#осществляем заполнения строк данных фрейма привлечения, данными из фрейма реатктивации
x = df_concat.groupby("date").count()
index = x.index

In [49]:
for date in index:
    count_users_rows(date, df_concat)

In [33]:
del data_calling
del data_reactivation

In [50]:
answer = df_concat.loc[df_concat['date'] == '2020-03-20']
answer_new = answer['users_count_new'].head(1).values[0]
answer_reactiv = answer['users_count_reactivated'].head(1).values[0]

In [51]:
print(f'количество привлечеённых пользователей 2020-03-20: {answer_new} ')
print(f'количество реактивированных пользователей 2020-03-20: {answer_reactiv}')

количество привлечеённых пользователей 2020-03-20: 5.0 
количество реактивированных пользователей 2020-03-20: 0.0


###### Все данные за 2020-03-20

In [52]:
answer 

,date,gmv360d_new,users_count_new,gmv360d_reactivated,users_count_reactivated
user_id,,,,,
26123607,2020-03-20,1.643711e+06,5.0,0.0,0.0
26576706,2020-03-20,1.643711e+06,5.0,0.0,0.0
30073680,2020-03-20,1.643711e+06,5.0,0.0,0.0
37559799,2020-03-20,1.643711e+06,5.0,0.0,0.0
40731603,2020-03-20,1.643711e+06,5.0,0.0,0.0


### Пример даты когда есть и привлчённые пользователи, и реактивированные

In [53]:
df_concat.loc[df_concat['date'] == '2020-04-20']

,date,gmv360d_new,users_count_new,gmv360d_reactivated,users_count_reactivated
user_id,,,,,
3458258,2020-04-20,681081.9551,2.0,3.247235e+06,14.0
4547859,2020-04-20,681081.9551,2.0,3.247235e+06,14.0
5745786,2020-04-20,681081.9551,2.0,3.247235e+06,14.0
11218662,2020-04-20,681081.9551,2.0,3.247235e+06,14.0
17474183,2020-04-20,681081.9551,2.0,3.247235e+06,14.0
29325807,2020-04-20,681081.9551,2.0,3.247235e+06,14.0
32441502,2020-04-20,681081.9551,2.0,3.247235e+06,14.0
32455188,2020-04-20,681081.9551,2.0,3.247235e+06,14.0
32845962,2020-04-20,681081.9551,2.0,3.247235e+06,14.0
